# [Semantic Search](https://huggingface.co/learn/nlp-course/chapter5/6?fw=pt)

In [1]:
# Built-in library
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pprint import pprint
import pandas as pd
from rich import print

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

### Load Data

In [2]:
import datasets
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict, Dataset

In [3]:
fp: str = "lewtun/github-issues"

issues_dataset: Dataset = load_dataset(fp, split="train")
issues_dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [4]:
print(issues_dataset[10])

{
    'url': 'https://api.github.com/repos/huggingface/datasets/issues/2944',
    'repository_url': 'https://api.github.com/repos/huggingface/datasets',
    'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/labels{/name}',
    'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/comments',
    'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/events',
    'html_url': 'https://github.com/huggingface/datasets/issues/2944',
    'id': 1000544370,
    'node_id': 'I_kwDODunzps47oxhy',
    'number': 2944,
    'title': 'Add  `remove_columns` to `IterableDataset ` ',
    'user': {
        'login': 'cccntu',
        'id': 31893406,
        'node_id': 'MDQ6VXNlcjMxODkzNDA2',
        'avatar_url': 'https://avatars.githubusercontent.com/u/31893406?v=4',
        'gravatar_id': '',
        'url': 'https://api.github.com/users/cccntu',
        'html_url': 'https://github.com/cccntu',
        'followers_url': 'https://api.github.com/users/cccntu/followers',
        'following_url': 'https://api.github.com/users/cccntu/following{/other_user}',
        'gists_url': 'https://api.github.com/users/cccntu/gists{/gist_id}',
        'starred_url': 'https://api.github.com/users/cccntu/starred{/owner}{/repo}',
        'subscriptions_url': 'https://api.github.com/users/cccntu/subscriptions',
        'organizations_url': 'https://api.github.com/users/cccntu/orgs',
        'repos_url': 'https://api.github.com/users/cccntu/repos',
        'events_url': 'https://api.github.com/users/cccntu/events{/privacy}',
        'received_events_url': 'https://api.github.com/users/cccntu/received_events',
        'type': 'User',
        'site_admin': False
    },
    'labels': [
        {
            'id': 1935892871,
            'node_id': 'MDU6TGFiZWwxOTM1ODkyODcx',
            'url': 'https://api.github.com/repos/huggingface/datasets/labels/enhancement',
            'name': 'enhancement',
            'color': 'a2eeef',
            'default': True,
            'description': 'New feature or request'
        }
    ],
    'state': 'open',
    'locked': False,
    'assignee': None,
    'assignees': [],
    'milestone': None,
    'comments': [],
    'created_at': 1632110460000,
    'updated_at': 1632110460000,
    'closed_at': None,
    'author_association': 'CONTRIBUTOR',
    'active_lock_reason': None,
    'pull_request': None,
    'body': '**Is your feature request related to a problem? Please describe.**\r\nA clear and concise description 
of what the problem is.\r\n\r\n```python\r\nfrom datasets import load_dataset\r\ndataset = load_dataset("c4", 
\'realnewslike\', streaming =True, split=\'train\')\r\ndataset = 
dataset.remove_columns(\'url\')\r\n```\r\n```\r\nAttributeError: \'IterableDataset\' object has no attribute 
\'remove_columns\'\r\n```\r\n\r\n**Describe the solution you\'d like**\r\n\r\nIt would be nice to have 
`.remove_columns()` to match the `Datasets` api. \r\n\r\n\r\n**Describe alternatives you\'ve 
considered**\r\n\r\nThis can be done with a single call to `.map()`, \r\n\r\nI can try to help add this. 🤗',
    'timeline_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/timeline',
    'performed_via_github_app': None,
    'is_pull_request': False
}

In [5]:
print(issues_dataset.column_names)

[
    'url',
    'repository_url',
    'labels_url',
    'comments_url',
    'events_url',
    'html_url',
    'id',
    'node_id',
    'number',
    'title',
    'user',
    'labels',
    'state',
    'locked',
    'assignee',
    'assignees',
    'milestone',
    'comments',
    'created_at',
    'updated_at',
    'closed_at',
    'author_association',
    'active_lock_reason',
    'pull_request',
    'body',
    'timeline_url',
    'performed_via_github_app',
    'is_pull_request'
]

In [6]:
# The issues_dataset contains issues and pull requests.
# Select ONLY the issues
issues_dataset_1: Dataset = issues_dataset.filter(
    (lambda x: x.get("is_pull_request") == False and len(x.get("comments")) > 0),
)

print(f"Size if data BEFORE: {issues_dataset.num_rows}\n")

print(f"Size if data AFTER: {issues_dataset_1.num_rows}\n")

Size if data BEFORE: 3019

Size if data AFTER: 808

In [7]:
all_columns: list[str] = issues_dataset_1.column_names
columns_to_keep: list[str] = ["title", "body", "html_url", "comments"]
# Columns present in ONLY all_columns
columns_to_remove: set = set(columns_to_keep).symmetric_difference(set(all_columns))
print(columns_to_remove)

{
    'events_url',
    'updated_at',
    'user',
    'created_at',
    'comments_url',
    'performed_via_github_app',
    'repository_url',
    'labels_url',
    'assignees',
    'author_association',
    'id',
    'active_lock_reason',
    'milestone',
    'labels',
    'number',
    'is_pull_request',
    'assignee',
    'url',
    'locked',
    'state',
    'pull_request',
    'node_id',
    'closed_at',
    'timeline_url'
}

In [8]:
issues_dataset_1 = issues_dataset_1.remove_columns(column_names=columns_to_remove)
issues_dataset_1

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [9]:
issues_dataset_1.set_format("pandas")
df: pd.DataFrame = issues_dataset_1[:]
# OR df = issues_dataset_1.to_pandas()

df.head()

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"[Cool, I think we can do both :), @lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).]","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - ..."
1,https://github.com/huggingface/datasets/issues/2943,Backwards compatibility broken for cached datasets that use `.filter()`,"[Hi ! I guess the caching mechanism should have considered the new `filter` to be different from the old one, and don't use cached results from the old `filter`.\r\nTo avoid other users from having this issue we could make the caching differentiate the two, what do you think ?, If it's easy enough to implement, then yes please 😄 But this issue can be low-priority, since I've only encountered it in a couple of `transformers` CI tests., Well it can cause issue with anyone that updates `datasets` and re-run some code that uses filter, so I'm creating a PR, I just merged a fix, let me know if...","## Describe the bug\r\nAfter upgrading to datasets `1.12.0`, some cached `.filter()` steps from `1.11.0` started failing with \r\n`ValueError: Keys mismatch: between {'indices': Value(dtype='uint64', id=None)} and {'file': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'speaker_id': Value(dtype='int64', id=None), 'chapter_id': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None)}`\r\n\r\nRelated feature: https://github.com/huggingface/datasets/pull/2836\r\n\r\n:question: This is probably a `wontfix` bug, since it can be solved by simply cleaning the..."
2,https://github.com/huggingface/datasets/issues/2941,OSCAR unshuffled_original_ko: NonMatchingSplitsSizesError,[I tried `unshuffled_original_da` and it is also not working],"## Describe the bug\r\n\r\nCannot download OSCAR `unshuffled_original_ko` due to `NonMatchingSplitsSizesError`.\r\n\r\n## Steps to reproduce the bug\r\n\r\n```python\r\n>>> dataset = datasets.load_dataset('oscar', 'unshuffled_original_ko')\r\nNonMatchingSplitsSizesError: [{'expected': SplitInfo(name='train', num_bytes=25292102197, num_examples=7345075, dataset_name='oscar'), 'recorded': SplitInfo(name='train', num_bytes=25284578514, num_examples=7344907, dataset_name='oscar')}]\r\n```\r\n\r\n## Expected results\r\n\r\nLoading is successful.\r\n\r\n## Actual results\r\n\r\nLoading throws ab..."
3,https://github.com/huggingface/datasets/issues/2937,load_dataset using default cache on Windows causes PermissionError: [WinError 5] Access is denied,"[Hi @daqieq, thanks for reporting.\r\n\r\nUnfortunately, I was not able to reproduce this bug:\r\n```ipython\r\nIn [1]: from datasets import load_dataset\r\n ...: ds = load_dataset('wiki_bio')\r\nDownloading: 7.58kB [00:00, 26.3kB/s]\r\nDownloading: 2.71kB [00:00, ?B/s]\r\nUsing custom data configuration default\r\nDownloading and preparing dataset wiki_bio/default (download: 318.53 MiB, generated: 736.94 MiB, post-processed: Unknown size, total: 1.03 GiB) to C:\Users\username\.cache\huggingface\datasets\wiki_bio\default\\r\n1.1.0\5293ce565954ba965dada626f1e79684e98172d950371d266

In [10]:
print(df["comments"].iloc[0])

['Cool, I think we can do both :)'
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen 
to protect the master branch only from **merge commits** (see update comment above), so no need to 
disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to 
the remote master branch; and eventually reverted without messing up the repo history).']

In [11]:
# There are two comments in this particular comment index
print(len(df["comments"].iloc[0]))

2

In [12]:
# Use the explode method to create a row for each comment in a given index.
# i.e. index 0 with 2 comments creates 2 rows and index 1 with 6 comments creates 6 rows, etc.
comments_df: pd.DataFrame = df.explode("comments", ignore_index=True)
comments_df.head(3)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"Cool, I think we can do both :)","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - ..."
1,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - ..."
2,https://github.com/huggingface/datasets/issues/2943,Backwards compatibility broken for cached datasets that use `.filter()`,"Hi ! I guess the caching mechanism should have considered the new `filter` to be different from the old one, and don't use cached results from the old `filter`.\r\nTo avoid other users from having this issue we could make the caching differentiate the two, what do you think ?","## Describe the bug\r\nAfter upgrading to datasets `1.12.0`, some cached `.filter()` steps from `1.11.0` started failing with \r\n`ValueError: Keys mismatch: between {'indices': Value(dtype='uint64', id=None)} and {'file': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'speaker_id': Value(dtype='int64', id=None), 'chapter_id': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None)}`\r\n\r\nRelated feature: https://github.com/huggingface/datasets/pull/2836\r\n\r\n:question: This is probably a `wontfix` bug, since it can be solved by simply cleaning the..."


In [13]:
# Convert back to Dataset
comments_dataset: Dataset = Dataset.from_pandas(comments_df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [14]:
# Add n new columns
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x.get("comments").split())}
)

comments_dataset

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2964
})

In [15]:
# Short comments
comments_dataset.sort("comment_length").select_columns(["comments", "comment_length"])[
    :10
]

{'comments': ['https://github.com/huggingface/datasets/blob/6c766f9115d686182d76b1b937cb27e099c45d68/src/datasets/builder.py#L179-L186',
  'https://github.com/huggingface/datasets/blob/6c766f9115d686182d76b1b937cb27e099c45d68/src/datasets/builder.py#L179-L186',
  '@albertvillanova ',
  'Thanks!',
  '#self-assign',
  '#take',
  '#take',
  '#self-assign',
  'Resolved',
  'Ty!'],
 'comment_length': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
# Drop short comments.
# i.e comments like: 'Thanks!', '#self-assign', etc.
comments_dataset = comments_dataset.filter(lambda x: x.get("comment_length") > 15)
comments_dataset

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})

In [23]:
# Concatenate the issue title, description (body), and comments together in a new text column.
def concat_data(example: dict[str, Any]) -> dict[str, Any]:
    """This is used to concatenate the title, body and comments together."""
    title: str = example.get("title")
    comments: str = example.get("comments")
    body: str = example.get("body")
    result = {"text": f"{title} \n {body} \n {comments}"}

    return result


comments_dataset = comments_dataset.map(concat_data)
comments_dataset

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text'],
    num_rows: 2175
})

In [24]:
comments_dataset[0]

{'html_url': 'https://github.com/huggingface/datasets/issues/2945',
 'title': 'Protect master branch',
 'comments': '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).',
 'body': 'After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pul

## Create Text Embeddings

- This [table](https://www.sbert.net/docs/pretrained_models.html#model-overview) shows the model overview for open source semantic search.

<br>

[![image.png](https://i.postimg.cc/MZLLL8TS/image.png)](https://postimg.cc/c6QTK2w9)

In [26]:
from transformers import AutoModel, AutoTokenizer


checkpoint: str = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

# Instantiate tokenizer
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Instantiate model
model: AutoModel = AutoModel.from_pretrained(checkpoint)

In [30]:
import torch

# To speed up the processing, push to a GPU
device_str: str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device=device_str)

device

device(type='cpu')

In [31]:
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [33]:
# To represent each GitHub issue entry as a single vector, we need to pool or
# average the token embeddings. One popular approach is CLS pooling, where the last
# hidden state for the special [CLS] token is collected.


def cls_pooling(model_output):
    """To represent each GitHub issue entry as a single vector, pool or average the token embeddings.
    Using CLS pooling, where the last hidden state for the special [CLS] token is collected."""
    return model_output.last_hidden_state[:, 0]


# Create a helper function that will tokenize a list of documents, place the tensors
# on the GPU (if available), feed them to the model, and finally apply CLS pooling to the outputs:


def get_embeddings(text_list: list):
    """This is used to tokenize a list of documents, place the tensors
    on the GPU (if available), feed them to the model, and apply CLS pooling to the outputs."""
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    # Push to encoded input to the GPU (if available)
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [35]:
print(comments_dataset["text"][0])

Protect master branch 
 After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all 
commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:
- 00cc036fea7c7745cfe722360036ed306796a3f2
- 13ae8c98602bbad8197de3b9b425f4c78f582af1
- ...

I propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the 
future:
-  For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is 
merged into the master branch
  - Currently, simple merge commits are already disabled
  - I propose to disable rebase merging as well
- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~
  - ~~This protection would reject direct pushes to master branch~~
  - ~~If so, for each release (when we need to commit directly to the master branch), we should previously disable 
the protection and re-enable it again after the release~~
-  Protect the master branch only from direct pushing of **merge commits**
  - GitHub offers the possibility to protect the master branch only from merge commits (which are the ones that 
introduce all the commits from the feature branch into the master branch).
  - No need to disable/re-enable this protection on each release 

This purpose of this Issue is to open a discussion about this problem and to agree in a solution. 
 @lhoestq now the 2 are implemented.

Please note that for the the second protection, finally I have chosen to protect the master branch only from 
**merge commits** (see update comment above), so no need to disable/re-enable the protection on each release 
(direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted 
without messing up the repo history).

In [36]:
# Test that the function works by feeding it the first text entry in our corpus
# and inspecting the output shape;
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [37]:
embedding

tensor([[-1.5532e-01, -1.0023e-01, -7.0321e-02, -7.9816e-02, -1.0425e-01,
         -1.8799e-01,  1.0403e-02,  2.7286e-01, -9.8491e-03, -8.4140e-02,
          2.9261e-01, -7.7875e-02, -1.3246e-01,  2.1589e-01, -5.6229e-02,
          1.7055e-01,  2.0032e-01, -4.0798e-02, -9.2333e-02,  2.7923e-02,
          2.2609e-02, -6.8703e-02,  8.3200e-02,  4.3630e-02, -1.8378e-01,
          5.4009e-03, -1.4811e-02,  1.5650e-01, -4.3977e-01, -4.6957e-01,
          1.4525e-01,  2.3224e-01,  2.5106e-02,  5.7053e-01, -1.0436e-04,
         -3.4580e-04,  1.6949e-01, -4.0601e-02, -1.4010e-01, -2.1546e-01,
         -4.9011e-01, -4.4480e-01, -7.0063e-02, -8.3547e-03,  1.2881e-01,
          5.2868e-02, -1.3250e-01,  2.3846e-02,  3.8485e-01,  1.0555e-01,
          2.3861e-01, -1.0285e-01,  1.8124e-01,  1.3770e-02,  3.9201e-01,
          5.4745e-01, -6.6635e-02,  3.0320e-01,  5.4018e-02,  6.7668e-04,
          6.1106e-02,  1.7411e-01, -8.2373e-02, -2.8667e-01,  2.2702e-01,
         -1.7799e-01,  5.1817e-01, -4.

In [39]:
# Get embeddings for the entire dataset
comments_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x.get("text")).detach().cpu().numpy()[0]}
)

comments_dataset

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)